http://mapfrappe.com/?show=51352

In [1]:
import time
import planarity
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import osmnx as ox
from shapely.geometry import Point, Polygon, LineString, MultiLineString

from planar import *

ox.config(use_cache=True, log_console=True)
%matplotlib inline

## Configure the script

In [2]:
df_places = pd.read_csv('data/places.csv')

distances = [805] #804.67 meters in each direction makes for 1 square mile, 500 meters is 1 sq km

network_types = ['drive', 'walk']

In [3]:
#df_places = df_places.loc[0:6]
df_places

,country,city,lat,lng
0,Argentina,Buenos Aires,-34.608657,-58.375547
1,Australia,Sydney,-33.863616,151.208977
2,Brazil,Sao Paulo,-23.549155,-46.633600
3,Canada,Toronto,43.649903,-79.380838
4,Canada,Vancouver,49.281651,-123.121363
5,Chile,Santiago,-33.439754,-70.654296
6,China,Beijing,39.908427,116.465505
7,China,Shanghai,31.236286,121.503787
8,China,Hong Kong,22.282321,114.156843
9,Denmark,Copenhagen,55.679443,12.578284


## Execute the script

In [4]:
# convert df to dict for execution
places = {}
for label, row in df_places.iterrows():
    key = '{}_{}'.format(row['country'], row['city'])
    value = (row['lat'], row['lng'])
    places[key] = value
    
places = df_places

In [5]:
results = []
for label, row in df_places.iterrows():
    
    coords = (row['lat'], row['lng'])
    point = Point(row['lng'], row['lat'])
    
    for network_type in network_types:
        
        for distance in distances:

            print(row['country'], row['city'], network_type, distance, end=' ')
            try:

                start_time = time.time()
                G = get_graph(coords, distance, network_type)

                # get a special buffered graph for planar analysis
                buffer = 500
                G_buff = get_graph(coords, distance + buffer, network_type)

                # get a bounding box to trim things square
                north, south, east, west = ox.bbox_from_point(coords, distance)
                bbox = Polygon([(west, north), (west, south), (east, south), (east, north)])

                # how many planar line intersections are there?
                planar_intersections = calculate_planar_intersections(G_buff, bbox)
                count_planar_intersections = len(planar_intersections)

                # how many nonplanar graph edge intersections are there?
                nonplanar_intersections = calculate_nonplanar_intersections(G, bbox)
                count_nonplanar_intersections = len(nonplanar_intersections)

                # how many cleaned, clustered intersections are there?
                cleaned_intersections = calculate_cleaned_intersections(nonplanar_intersections, G.graph['crs'])
                count_cleaned_intersections = len(cleaned_intersections)
                
                # how does planarity affect average edge length?
                mean_edge_length, mean_planar_segment_length, edge_length_ratio = calculate_edge_length_ratios(G, planar_intersections)

                if count_nonplanar_intersections > 0:
                    # planar line intersections overcounts nonplanar graph edge intersections by xx%
                    # ie, planar graph shows xx% more intersections than nonplanar graph with bridges/tunnels
                    planar_nonplanar_overcount = count_planar_intersections / count_nonplanar_intersections
                else:
                    planar_nonplanar_overcount = None #avoid divide by zero errors

                if count_cleaned_intersections > 0:
                    # edge intersections overcounts street intersections by xx%
                    nonplanar_cleaned_overcount = count_nonplanar_intersections / count_cleaned_intersections

                    # line intersections overcounts street intersections by xx%
                    planar_cleaned_overcount = count_planar_intersections / count_cleaned_intersections
                else:
                    planar_cleaned_overcount = None #avoid divide by zero errors
                    nonplanar_cleaned_overcount = None

                # is it a formally planar graph (ignoring spatial embedding)?
                warnings.filterwarnings('ignore')
                is_planar = planarity.is_planar(G)
                warnings.resetwarnings()
                
                runtime = round(time.time() - start_time, 2)
                print(runtime)
                
                # assemble the results
                result = {'country' : row['country'], 
                          'city' : row['city'],
                          'geometry' : point,
                          'distance' : distance,
                          'network_type' : network_type,
                          'nodes' : len(G.nodes()),
                          'count_planar_intersections' : count_planar_intersections,
                          'count_nonplanar_intersections' : count_nonplanar_intersections,
                          'count_cleaned_intersections' : count_cleaned_intersections,
                          'overcount_planar_nonplanar' : planar_nonplanar_overcount,
                          'overcount_nonplanar_cleaned' : nonplanar_cleaned_overcount,
                          'overcount_planar_cleaned' : planar_cleaned_overcount,
                          'mean_edge_length' : mean_edge_length,
                          'mean_planar_segment_length' : mean_planar_segment_length,
                          'edge_length_ratio' : edge_length_ratio,
                          'phi' : 1 / planar_nonplanar_overcount,
                          'is_planar' : is_planar,
                          'runtime' : runtime}
                results.append(result)

            except Exception as e:
                print(e)

Argentina Buenos Aires drive 805 8.21
Argentina Buenos Aires walk 805 

g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\core.py:112: ResourceWarning: unclosed file <_io.TextIOWrapper name='cache/f02ccae7041ed2992dd986d557f2973a.json' mode='r' encoding='utf-8'>
  response_json = json.load(io.open(cache_path_filename, encoding='utf-8'))
g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\core.py:112: ResourceWarning: unclosed file <_io.TextIOWrapper name='cache/08e26ad7b03cbe0aa62de7d90f7d27e4.json' mode='r' encoding='utf-8'>
  response_json = json.load(io.open(cache_path_filename, encoding='utf-8'))


63.17
Australia Sydney drive 805 

g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\core.py:112: ResourceWarning: unclosed file <_io.TextIOWrapper name='cache/78c0ce84465289c5b79fea7cbe961c4c.json' mode='r' encoding='utf-8'>
  response_json = json.load(io.open(cache_path_filename, encoding='utf-8'))
g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\core.py:112: ResourceWarning: unclosed file <_io.TextIOWrapper name='cache/2975128ec65f4e97082a57bb1beff199.json' mode='r' encoding='utf-8'>
  response_json = json.load(io.open(cache_path_filename, encoding='utf-8'))


5.6
Australia Sydney walk 805 

...

226.47
Venezuela Caracas drive 805 

g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\core.py:112: ResourceWarning: unclosed file <_io.TextIOWrapper name='cache/760316881960c213b9b2ba29ec3a14bd.json' mode='r' encoding='utf-8'>
  response_json = json.load(io.open(cache_path_filename, encoding='utf-8'))
g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\core.py:112: ResourceWarning: unclosed file <_io.TextIOWrapper name='cache/e8b558e7b73dd649452c94099819375f.json' mode='r' encoding='utf-8'>
  response_json = json.load(io.open(cache_path_filename, encoding='utf-8'))


7.67
Venezuela Caracas walk 805 

g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\core.py:112: ResourceWarning: unclosed file <_io.TextIOWrapper name='cache/b61e7e55a4c1a2eab70fc54918034254.json' mode='r' encoding='utf-8'>
  response_json = json.load(io.open(cache_path_filename, encoding='utf-8'))
g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\core.py:112: ResourceWarning: unclosed file <_io.TextIOWrapper name='cache/47cf0803991bf286fb64c6bbbf25f1d8.json' mode='r' encoding='utf-8'>
  response_json = json.load(io.open(cache_path_filename, encoding='utf-8'))


10.66


In [6]:
gdf = gpd.GeoDataFrame(results, geometry='geometry').round(3)
gdf.crs = {'init': 'epsg:4326'}
print('finished in {:.2f} minutes'.format(gdf['runtime'].sum() / 60))

finished in 118.58 minutes


## Save results to disk

In [14]:
ox.save_graph_shapefile(G, filename='graph', folder='data')

In [15]:
planar_intersections.to_file('data/graph/planar')
nonplanar_intersections.to_file('data/graph/nonplanar')
cleaned_intersections.to_file('data/graph/cleaned')

In [16]:
north, south, east, west = ox.bbox_from_point(coords, distance)
polygon = Polygon([(west, north), (west, south), (east, south), (east, north)])
gpd.GeoSeries([polygon]).to_file('data/graph/bbox')

In [17]:
gdf.to_csv('data/results.csv', index=True, encoding='utf-8')

In [18]:
gdf2 = gdf.set_index(['country', 'city', 'distance', 'network_type'])
gdf2.iloc[gdf2.index.get_level_values('network_type') == 'drive'].to_csv('data/results_drive.csv')
gdf2.iloc[gdf2.index.get_level_values('network_type') == 'walk'].to_csv('data/results_walk.csv')